### 2.2 The Hierarchical PALM4MSA based FGFT Algorithms

To finish this notebook briefly, we won't go into further details about the second algorithm for computing approximate FGFTs because it's an extension of the algorithms discussed in [1](#1.-The-Hierarchical-PALM4MSA-Algorithm) and besides this algorithm is considered experimental (and therefore is not included in pyfaust release).

However, you'll find below a copy of the main example of [fgft_palm()](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/namespacepyfaust_1_1fact.html#a82967b7913a66c945f32b7bd1ec6625a). This function is the entry point of the algorithm and again it's very similar to what you've seen above.
You can try to unroll it progressively and understand how it tries to do the same thing as the eigtj() on another Laplacian matrix.
It's much more complicated to configure but starting from an example you'll be able to write your own FGFT calculation.

About the Laplacian matrix used in the code, note that pyfaust embeds some data examples (matrices in matlab v7 format) but depending on how you installed the package you may need to download them. It's not a big deal normally because ```get_data_dirpath()``` should handle it automatically (otherwise please read the [help doc](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/FAQ.html)). Note that depending on your connection the download can take some time.

In [ ]:
from pyfaust.fact import fgft_palm
from pyfaust.factparams import *
from pyfaust.demo import get_data_dirpath
from os.path import sep
from scipy.io import loadmat, savemat
from numpy.linalg import eig, eigh, norm
from numpy import sort, argsort, log2, size, copy, diag
d = loadmat(sep.join((get_data_dirpath(),'Laplacian_256_ring.mat')))
Lap = d['Lap'].astype('float')
D, U = eig(Lap)
indices = argsort(D)
D = D[indices].astype('float')
U = U[:,indices].astype('float')
print(D.shape, type(D))
print("eig(Lap), U error:", norm(Lap.dot(U)-U.dot(diag(D))))
dim = Lap.shape[0]
nfacts = int(round(log2(dim))-3)
over_sp = 1.5 # sparsity overhead
dec_fact = .5 # decrease of the residum sparsity
fact_cons, res_cons = [], []
for j in range(1, nfacts):
        fact_cons += [ ConstraintInt('sp',dim,dim,
        min(int(round(dec_fact**j*dim**2*over_sp)),size(Lap)))
    ]
        res_cons += [
                ConstraintInt('sp',
            dim,
            dim,
            min(int(round(2*dim*over_sp)),size(Lap)))
        ]
params = ParamsHierarchical(fact_cons,
        res_cons,
        StoppingCriterion(num_its=50),
        StoppingCriterion(num_its=100),
        step_size=1.0000e-06,
        constant_step_size=True,
        init_lambda=1.0,
        is_fact_side_left=False)
Lap = Lap.astype(float)
Uhat,Dhat = fgft_palm(Lap, U, params, init_D=D)
err_U = (Uhat-U).norm()/norm(U)
err_Lap = norm(Uhat.toarray()@diag(Dhat)@Uhat.T.toarray()-Lap)/norm(Lap)
print(norm(diag(Dhat), 2))
print("err_U:", err_U)